# Data augmentation

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import pickle

params = {'axes.grid': True,
          'grid.linestyle': '--',
          }
plt.rcParams.update(params)

In [ ]:
with open('data/tt_epi.pkl', 'rb') as f:
    myo, period_range = pickle.load(f)

myo = myo[:, 2::4]
myo -= myo.min(axis=1, keepdims=True)
myo /= myo.max()
myo = np.pad(myo, ((0, 0), (1, 0)), 'constant', constant_values=0)[:,:-1] # move 1 sample to the right

In [ ]:
# Scale the data

ds = [myo, myo*1.1, myo*0.9, myo*0.8]

ext = myo.copy()
ext[:, 26] = 1
ext[:, 27] = 0.8
ds.append(ext)

ext = myo.copy() * 0.9
ext[:, 26] = 1
ext[:, 27] = 0.8
ds.append(ext)

ext = myo.copy() * 1.1
ext[:, 26] = 1
ext[:, 27] = 0.8
ds.append(ext)

ext = myo.copy() * 0.8
ext[:, 26] = 1
ext[:, 27] = 0.8
ds.append(ext)

ds = np.concatenate(ds, axis=0)
ds.shape

In [ ]:
for _m in ds[::10]:
    plt.plot(_m, 'k', alpha=0.1)
# plt.xlim(24, 30)

In [ ]:
with open('data/tt_epi_ds.pkl', 'wb') as f:
    pickle.dump(ds, f)

In [ ]:
# Split and interpolate

ext = []
for yp in ds:
    xp = np.linspace(0, 1, len(yp))

    cutoff_1 = 28
    max_1 = cutoff_1 + np.argmax(yp[cutoff_1:])

    for ratio in np.linspace(1.1, 1.5, 10):

        x = np.linspace(0, 1, int(len(yp)*ratio))

        max_2 = int(max_1 * ratio)

        y = np.interp(x, xp, yp, left=0, right=0)

        glued = np.concatenate((yp[:max_1], y[max_2:][:len(xp[max_2:])]))
        glued = np.pad(glued, (0, 250-len(glued)))
        ext.append(glued)
ext = np.array(ext)
ds = np.concatenate((ds, ext), axis=0)

In [ ]:
with open('data/tt_epi_ds_long.pkl', 'wb') as f:
    pickle.dump(ds, f)